In [1]:
import numpy as np  
import pandas as pd
import re
from string import ascii_letters

from keras.models import Sequential
from keras.layers import LSTM,Flatten,Dense
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint

from WCPre import PreSentence2Vec
from nltk.corpus import stopwords

file_path = "./weights/w2c_LSTM-L20-L10-dim50-e{epoch:02d}-loss{loss:.4f}.hdf5"

Using TensorFlow backend.


In [2]:
# Read data from csv file 
File = 'Data/train.csv'
dataSet = pd.read_csv(File)
del dataSet['id']
X = dataSet['comment_text']
X = X.values
Y = dataSet.T.drop('comment_text').T
Y = Y.values

In [11]:
# X = X[1:100]
# Y = Y[1:100]

In [4]:
for i in range(0,len(X)):
    # Remove \\n Convert I'm --> I am etc \n",
    comment = X[i]
    comment = PreSentence2Vec.preProcessing(comment)
    # Remove the link & numbers \n",
    comment = PreSentence2Vec.preNumberLink(comment)
    # Remove the punctuation\n",
    comment = PreSentence2Vec.prePunctuation(comment)
    # Captital --> lowercase --> wordlist \n",
    comment = comment.lower()
    comment = re.sub("[^\w]", " ",  comment).split()
    # Remove stopwords\n",
    comment = [word for word in comment if word not in stopwords.words('english')]
    # Remove Non-English"
    character = set(ascii_letters)
    comment = [word for word in comment if any(letter in character for letter in word)]
    
    X[i] = ' '.join(comment)

In [5]:
print("finish preprocessing")

finish preprocessing


In [7]:
# Word Embedding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
print("Find %s in training sets" % len(word_index))

maxLength = max(max(X, key=len))
X = pad_sequences(X, maxlen=maxLength)

X_train = X[0:round(len(X)*0.9)]
X_validate = X[round(len(X)*0.9):]
Y_train = Y[0:round(len(Y)*0.9)]
Y_validate = Y[round(len(Y)*0.9):]

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [144]:
# Splite it into Training & Testing data

# load word embeddings for the embedding layer
embeddingDIM = 50
embeddingsRoute = "./Embedding/glove.twitter.%s.txt" % embeddingDIM
embeddings_index = {}
f = open(embeddingsRoute, 'rt', encoding='utf8')
print("Load embedding matrix")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, embeddingDIM))

# Word --> Vector
count = 0
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        count += 1 

print(count)

Load embedding matrix


Found 1193514 word vectors.
1696


In [146]:
model = Sequential()
model.add(
    Embedding(len(word_index) + 1, embeddingDIM, weights=[embedding_matrix], input_length=maxLength, trainable=False))
model.add(LSTM(24, dropout=0.01, recurrent_dropout=0.01, use_bias=True,
                   return_sequences=True))
model.add(LSTM(12, dropout=0.01, recurrent_dropout=0.01, use_bias=True,
                   return_sequences=True))
model.add(Flatten())
model.add(Dense(Y.shape[1],activation='softmax', use_bias=True))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

print(model.summary())

callbacks = [
    # Metrics(),
    # LearningRateScheduler(step_decay),
    # EarlyStopping(monitor="val_acc", patience=5, verbose=1),
    # ReduceLROnPlateau(monitor=monitor_value, factor=0.2, patience=2, min_lr=0.001),
    ModelCheckpoint(file_path, monitor='val_loss', save_best_only=False, period=1, verbose=1),
]
model.fit(X_train, Y_train, validation_data=(X_validate, Y_validate), batch_size=128,
              initial_epoch=0, epochs=100, shuffle=False,
              verbose=1, callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1345, 50)          90700     
_________________________________________________________________
lstm_3 (LSTM)                (None, 1345, 24)          7200      
_________________________________________________________________
lstm_4 (LSTM)                (None, 1345, 12)          1776      
_________________________________________________________________
flatten_2 (Flatten)          (None, 16140)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 96846     
Total params: 196,522
Trainable params: 105,822
Non-trainable params: 90,700
_________________________________________________________________
None


Train on 89 samples, validate on 10 samples
Epoch 1/100


Epoch 00001: saving model to ./weights/w2c_LSTM-L20-L10-dim50-e01-loss0.6670.hdf5
89/89 [==============================] - 14s 160ms/step - loss: 0.6670 - acc: 0.0000e+00 - val_loss: 0.0000e+00 - val_acc: 0.4000


Epoch 2/100


Epoch 00002: saving model to ./weights/w2c_LSTM-L20-L10-dim50-e02-loss0.6499.hdf5
89/89 [==============================] - 11s 129ms/step - loss: 0.6499 - acc: 0.4719 - val_loss: 0.0000e+00 - val_acc: 0.5000


...

Epoch 9/100


Epoch 00009: saving model to ./weights/w2c_LSTM-L20-L10-dim50-e09-loss0.5317.hdf5
89/89 [==============================] - 12s 132ms/step - loss: 0.5317 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000


Epoch 10/100


Epoch 00010: saving model to ./weights/w2c_LSTM-L20-L10-dim50-e10-loss0.5380.hdf5
89/89 [==============================] - 12s 140ms/step - loss: 0.5380 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000


Epoch 11/100
